In [ ]:
%pip install -q transformers tensorflow tf-keras tensorflow-datasets

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
from tqdm import tqdm
import tensorflow_datasets as tfds

In [ ]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    as_supervised=True,
    with_info=True
)

In [ ]:
from transformers import BertTokenizer, TFBertModel

cache_dir = "model_cache"

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir=cache_dir, download=True)
model = TFBertModel.from_pretrained('bert-base-uncased', cache_dir=cache_dir, download=True)


In [ ]:
def generate_bert_embeddings(data, tokenizer, model):
    embeddings = []
    for sentence, _ in tqdm(data):
        inputs = tokenizer(sentence, return_tensors="tf", padding=True, truncation=True, max_length=128)
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        sentence_embedding = tf.reduce_mean(last_hidden_states, axis=1).numpy()
        embeddings.append(sentence_embedding)
    return np.vstack(embeddings)

train_embeddings = generate_bert_embeddings(train_data, tokenizer, model)
test_embeddings = generate_bert_embeddings(test_data, tokenizer, model)
